In [48]:
import read_resume
import File_downloader_from_github as file_downloader
import re
import pandas as pd
from collections import Counter

In [ ]:
company_csv_preview = pd.read_csv("Datasets/Companies_Dataset.csv")
university_csv_preview = pd.read_csv("Datasets/Universities_Dataset.csv")
skills_list_preview = pd.read_csv("Datasets/Skills_Dataset.csv")

                    Name
0                 Apple 
1     NVIDIA Corporation
2  Microsoft Corporation
3                Amazon 
4                Google 
   ranking                             University        Country  Impact Rank  \
0        1                     Harvard University  United States            1   
1        2                    Stanford University  United States            3   
2        3  Massachusetts Institute of Technology  United States            1   
3        4      University of California Berkeley  United States            4   
4        5                   University of Oxford             UK           15   

   Openness Rank  Excellence Rank  
0              1                1  
1              2                4  
2              3               15  
3              4               27  
4              5                5  
            Job_Role       Skill  Score  \
0  Software Engineer      Python    9.7   
1  Software Engineer        Java    9.6   
2  Software Engine

In [ ]:
print(company_csv_preview.head())
print(university_csv_preview.head())
print(skills_list_preview.head())

In [15]:
company_csv_preview.to_csv('Datasets/Companies_Dataset.csv')

In [22]:
company_csv_preview = pd.read_csv('Datasets/Companies_Dataset.csv')
print(company_csv_preview.head())

   Unnamed: 0                   Name
0           0                 Apple 
1           1     NVIDIA Corporation
2           2  Microsoft Corporation
3           3                Amazon 
4           4                Google 


In [25]:
company_csv_preview = company_csv_preview.rename(columns = {"Unnamed: 0" : 'Rank'})
company_csv_preview['Rank'] += 1
print(company_csv_preview.head())

   Rank                   Name
0     1                 Apple 
1     2     NVIDIA Corporation
2     3  Microsoft Corporation
3     4                Amazon 
4     5                Google 


In [34]:
company_csv_preview.to_csv('Datasets/Companies_Dataset.csv', index = False)

University Dataset Preprocessing

In [ ]:
university_csv_preview.drop(columns = ['Unnamed: 0', 'Country','Impact Rank','Openness Rank','Excellence Rank'], inplace = True)
print(university_csv_preview.head())

   ranking                             University
0        1                     Harvard University
1        2                    Stanford University
2        3  Massachusetts Institute of Technology
3        4      University of California Berkeley
4        5                   University of Oxford


In [33]:
university_csv_preview.to_csv('Datasets/Universities_Dataset.csv', index = False)

In [35]:
print(company_csv_preview.head())
print(university_csv_preview.head())
print(skills_list_preview.head())

   Rank                   Name
0     1                 Apple 
1     2     NVIDIA Corporation
2     3  Microsoft Corporation
3     4                Amazon 
4     5                Google 
   ranking                             University
0        1                     Harvard University
1        2                    Stanford University
2        3  Massachusetts Institute of Technology
3        4      University of California Berkeley
4        5                   University of Oxford
            Job_Role       Skill  Score  \
0  Software Engineer      Python    9.7   
1  Software Engineer        Java    9.6   
2  Software Engineer         C++    9.6   
3  Software Engineer  JavaScript    9.5   
4  Software Engineer  TypeScript    9.4   

                                  Position  
0  Intern, Junior, Mid-level, Senior, Lead  
1  Intern, Junior, Mid-level, Senior, Lead  
2  Intern, Junior, Mid-level, Senior, Lead  
3  Intern, Junior, Mid-level, Senior, Lead  
4          Junior, Mid-level, 

In [49]:
import read_resume
import File_downloader_from_github as file_downloader
import re
import pandas as pd
from collections import Counter

def load_rankings(company_csv, university_csv, skills_csv):
    """
    Loads company, university rankings, and skills from CSV files.
    """
    company_df = pd.read_csv(company_csv)
    university_df = pd.read_csv(university_csv)
    skills_df = pd.read_csv(skills_csv)
    
    # Normalize column names and values
    company_ranking = dict(zip(company_df['Name'].str.strip().str.lower(), company_df['Rank']))
    university_ranking = dict(zip(university_df['University'].str.strip().str.lower(), university_df['ranking']))
    skill_scores = dict(zip(skills_df['Skill'].str.strip().str.lower(), skills_df['Score']))
    
    return company_ranking, university_ranking, skill_scores

def get_tier_score(entity, ranking_dict, tier_mapping):
    """
    Assigns a score based on the entity's ranking.
    """
    rank = ranking_dict.get(entity, None)
    if rank is None:
        return 0  # Entity not ranked
    
    for tier, score in tier_mapping.items():
        if tier[0] <= rank <= tier[1]:
            return score
    return 0

def calculate_resume_score(resume_text, company_csv, university_csv, skills_csv, company_tiers, university_tiers):
    """
    Computes the weighted resume score based on extracted entities.
    """
    # Load rankings from CSV files
    company_ranking, university_ranking, skill_scores = load_rankings(company_csv, university_csv, skills_csv)
    
    # Tokenize resume text
    words = re.findall(r'\w+', resume_text.lower())
    word_counts = Counter(words)
    
    # Extract and score companies
    company_scores = [get_tier_score(company, company_ranking, company_tiers) for company in company_ranking if company in word_counts]
    
    # Extract and score universities
    university_scores = [get_tier_score(university, university_ranking, university_tiers) for university in university_ranking if university in word_counts]
    
    # Extract and score skills
    skill_scores_list = [skill_scores[skill] for skill in skill_scores if skill in word_counts]
    
    # Compute dynamic weights
    total_mentions = len(company_scores) + len(university_scores) + len(skill_scores_list)
    
    if total_mentions == 0:
        return 0  # No relevant data in resume
    
    company_weight = len(company_scores) / total_mentions
    university_weight = len(university_scores) / total_mentions
    skill_weight = len(skill_scores_list) / total_mentions
    
    # Compute final weighted score
    final_score = (
        sum(company_scores) * company_weight +
        sum(university_scores) * university_weight +
        sum(skill_scores_list) * skill_weight
    )
    
    return round(final_score, 2)

# Example tier mappings
company_tiers = {
    (1, 10): 100, (11, 50): 90, (51, 100): 80, (101, 1000): 70, 
    (1001, 5000): 60, (5001, 20000): 50, (20001, 50000): 40, (50001, 100000): 30, 
    (100001, 200000): 20, (200001, float('inf')): 10
}

university_tiers = {
    (1, 10): 100, (11, 50): 90, (51, 100): 80, (101, 250): 70, 
    (251, 500): 60, (501, 1000): 50, (1001, 1500): 40, (1501, 2000): 30, 
    (2001, 3000): 20, (3001, float('inf')): 10
}

if __name__ == "__main__":  
    resume_name = 'autoCV (2).pdf'
    pdf_path = "Resumes/" + resume_name
    # pdf_path = "Resume_Scrapper/Resumes/autoCV (1).pdf"
    resume_name = resume_name[:-4]

    resume_text, extracted_links = read_resume.extract_text_and_links_from_pdf(pdf_path)
    
    with open(resume_name + ".txt", "wb") as f:
        f.write(resume_text.encode('utf-8'))
    print(f"Downloaded: {resume_name}")
    
    for link in extracted_links:
        if "github" in link:
            file_downloader.Downloader(link)
    
    company_csv = "Companies_Dataset.csv"
    university_csv = "Universities_Dataset.csv"
    skills_csv = "Skills_Dataset.csv"
    
    final_resume_score = calculate_resume_score(resume_text, company_csv, university_csv, skills_csv, company_tiers, university_tiers)
    print("Final Resume Score:", final_resume_score)


Downloaded: autoCV (2)


FileNotFoundError: [Errno 2] No such file or directory: 'Resume_Scrapper/Downloaded/code_filesDisease_Prediction.ipynb'

In [ ]:
def load_rankings(company_csv, university_csv):
    """
    Loads company and university rankings from CSV files.
    """
    company_df = pd.read_csv(company_csv)
    university_df = pd.read_csv(university_csv)
    
    company_ranking = dict(zip(company_df['company'].str.lower(), company_df['rank']))
    university_ranking = dict(zip(university_df['university'].str.lower(), university_df['rank']))
    
    return company_ranking, university_ranking

In [ ]:
def get_tier_score(entity, ranking_dict, tier_mapping):
    """
    Assigns a score based on the entity's ranking.
    """
    rank = ranking_dict.get(entity, None)
    if rank is None:
        return 0  # Entity not ranked
    
    for tier, score in tier_mapping.items():
        if tier[0] <= rank <= tier[1]:
            return score
    return 0

In [ ]:
def calculate_resume_score(resume_text, company_csv, university_csv, skills_list, company_tiers, university_tiers):
    """
    Computes the weighted resume score based on extracted entities.
    """
    # Load rankings from CSV files
    company_ranking, university_ranking = load_rankings(company_csv, university_csv)
    
    # Tokenize resume text
    words = re.findall(r'\w+', resume_text.lower())
    word_counts = Counter(words)
    
    # Extract and score companies
    company_scores = [get_tier_score(company, company_ranking, company_tiers) for company in company_ranking if company in word_counts]
    
    # Extract and score universities
    university_scores = [get_tier_score(university, university_ranking, university_tiers) for university in university_ranking if university in word_counts]
    
    # Extract and score skills
    skill_scores = [10 for skill in skills_list if skill in word_counts]  # Simple fixed score per skill
    
    # Compute dynamic weights
    total_mentions = len(company_scores) + len(university_scores) + len(skill_scores)
    
    if total_mentions == 0:
        return 0  # No relevant data in resume
    
    company_weight = len(company_scores) / total_mentions
    university_weight = len(university_scores) / total_mentions
    skill_weight = len(skill_scores) / total_mentions
    
    # Compute final weighted score
    final_score = (
        sum(company_scores) * company_weight +
        sum(university_scores) * university_weight +
        sum(skill_scores) * skill_weight
    )
    
    return round(final_score, 2)

# Example tier mappings
company_tiers = {
    (1, 10): 100, (11, 50): 90, (51, 100): 80, (101, 1000): 70, 
    (1001, 5000): 60, (5001, 20000): 50, (20001, 50000): 40, (50001, 100000): 30, 
    (100001, 200000): 20, (200001, float('inf')): 10
}

university_tiers = {
    (1, 10): 100, (11, 50): 90, (51, 100): 80, (101, 250): 70, 
    (251, 500): 60, (501, 1000): 50, (1001, 1500): 40, (1501, 2000): 30, 
    (2001, 3000): 20, (3001, float('inf')): 10
}

In [ ]:
if __name__ == "__main__":  
    resume_name = 'autoCV (1).pdf'
    pdf_path = "Resume_Scrapper/Resumes/"+resume_name

    resume_text, extracted_links = read_resume.extract_text_and_links_from_pdf(pdf_path)
    
    # print(f"Extractd Text: {resume_text}")

    # print("Extracted Links:")
    # for link in extracted_links:
    #     print(link)
    # print("\n")

    with open("Resume_Scrapper/Downloaded/code_files"+resume_name, "wb") as f:
        f.write(resume_text)
    print(f"Downloaded: {resume_name}")
    
    for link in extracted_links:
        if "github" in link:
            file_downloader.Downloader(link)
    
    company_csv = "Companies_Dataset.csv"
    university_csv = "Universities_Dataset.csv"
    skills_list = ["python", "machine learning", "data science", "java"]

    final_resume_score = calculate_resume_score(resume_text, company_csv, university_csv, skills_list, company_tiers, university_tiers)
    print("Final Resume Score:", final_resume_score)